In [1]:
] activate .

In [4]:
using BenchmarkTools, LinearAlgebra, StaticArrays

# Simple Point

In [26]:
struct SimplePoint
    x
    y
    z
end

In [98]:
p = SimplePoint(1,2,3)

SimplePoint(1, 2, 3)

In [99]:
q = SimplePoint(0,1,0)

SimplePoint(0, 1, 0)

In [101]:
p + q

SimplePoint(1, 3, 3)

In [102]:
Base.:+(a::SimplePoint, b::SimplePoint) = SimplePoint(a.x+b.x, a.y+b.y, a.z+b.z)

In [103]:
p + q

SimplePoint(1, 3, 3)

In [104]:
@btime $(fill(p, 1000)) .+ $(fill(q, 1000));

  53.899 μs (1001 allocations: 39.19 KiB)


# Parametric Point

In [105]:
struct ParamPoint{T}
    x::T
    y::T
    z::T
end

Base.:+(a::ParamPoint, b::ParamPoint) = ParamPoint(a.x+b.x, a.y+b.y, a.z+b.z)

In [106]:
p = ParamPoint(1,2,3)
q = ParamPoint(0,1,0)

ParamPoint{Int64}(0, 1, 0)

In [107]:
p + q

ParamPoint{Int64}(1, 3, 3)

In [108]:
@btime $(fill(p, 1000)) .+ $(fill(q, 1000));

  1.867 μs (2 allocations: 23.52 KiB)


# N-dimensional Point

In [109]:
struct NPoint{T}
    cords::Vector{T}
end

NPoint(x::T...) where T = NPoint{T}([x...])

Base.:+(a::NPoint, b::NPoint) = NPoint(a.cords .+ b.cords)

In [110]:
p = NPoint(1,2,3)
q = NPoint(0,1,0)

NPoint{Int64}([0, 1, 0])

In [111]:
p + q

NPoint{Int64}([1, 3, 3])

In [112]:
@btime $(fill(p, 1000)) .+ $(fill(q, 1000));

  38.999 μs (2001 allocations: 132.94 KiB)


# N-dimensional Point (second try)

In [113]:
struct NPoint2{N, T}
    cords::NTuple{N, T}
end

NPoint2(x::T...) where T = NPoint2{length(x), T}(x)

Base.:+(a::NPoint2, b::NPoint2) = NPoint2(a.cords .+ b.cords)

In [114]:
p = NPoint2(1,2,3)
q = NPoint2(0,1,0)

NPoint2{3,Int64}((0, 1, 0))

In [115]:
p + q

NPoint2{3,Int64}((1, 3, 3))

In [116]:
@btime $(fill(p, 1000)) .+ $(fill(q, 1000));

  1.800 μs (2 allocations: 23.52 KiB)


This is just as efficient as our handwritten `ParamPoint` above but works for an arbitrary number of dimensions!

# N-dimensional mutable Point

In [117]:
mutable struct NPoint3{N, T}
    cords::NTuple{N, T}
end

NPoint3(x::T...) where T = NPoint3{length(x), T}(x)

Base.:+(a::NPoint3, b::NPoint3) = NPoint3(a.cords .+ b.cords)

In [118]:
p = NPoint3(1,2,3)
q = NPoint3(0,1,0)

NPoint3{3,Int64}((0, 1, 0))

In [119]:
p+q

NPoint3{3,Int64}((1, 3, 3))

In [120]:
@btime $(fill(p, 1000)) .+ $(fill(q, 1000));

  5.850 μs (1001 allocations: 39.19 KiB)


# StaticPoint

In [167]:
struct StaticPoint{N, T}
    cords::SVector{N, T} # MVector for a mutable version
end

StaticPoint(x::T...) where T = StaticPoint(SVector{length(x), T}(x))

StaticPoint

In [168]:
Base.:+(a::StaticPoint, b::StaticPoint) = StaticPoint(a.cords .+ b.cords)

In [169]:
p = StaticPoint(1,2,3)
q = StaticPoint(0,1,0)

StaticPoint{3,Int64}([0, 1, 0])

In [170]:
p+q

StaticPoint{3,Int64}([1, 3, 3])

In [171]:
@btime $(fill(p, 1000)) .+ $(fill(q, 1000));

  1.822 μs (2 allocations: 23.52 KiB)


# Function forwarding

In [134]:
using Lazy: @forward

In [139]:
@forward StaticPoint.cords LinearAlgebra.norm

In [140]:
norm(p)

3.7416573867739413

In [157]:
@forward StaticPoint.cords (Base.getindex, Base.length, Base.size, Base.eltype)

In [153]:
p[1]

1

In [154]:
p[1:3]

3-element Array{Int64,1}:
 1
 2
 3

In [155]:
length(p)

3

In [156]:
size(p)

(3,)

In [158]:
eltype(p)

Int64

In [159]:
@forward StaticPoint.cords Base.iterate

In [173]:
for x in q
    @show x
end

x = 0
x = 1
x = 0
